In [12]:
import numpy as np 
import pandas as pd
import http.client
from bs4 import BeautifulSoup, element
import requests
#Datasetimi okudum
df = pd.read_csv("D:\indirenler 1\mülakat ödevi/api_veri.csv")

In [13]:
df

,id,date,conract,price,quantity
0,444121195,2022-01-26T00:00:34+03:00,PH22012603,731.99,5
1,444121022,2022-01-26T00:00:35+03:00,PH22012603,732.00,5
2,444121234,2022-01-26T00:00:43+03:00,PH22012603,731.99,5
3,444120877,2022-01-26T00:00:56+03:00,PH22012608,1341.99,10
4,444121289,2022-01-26T00:00:56+03:00,PH22012608,1341.98,10
...,...,...,...,...,...
11137,445668087,2022-01-26T23:58:36+03:00,PH22012701,1375.02,17
11138,445668462,2022-01-26T23:58:48+03:00,PH22012707,1371.01,1
11139,445668795,2022-01-26T23:58:50+03:00,PH22012708,1370.40,1
11140,445669965,2022-01-26T23:59:28+03:00,PH22012704,1340.01,18


In [17]:
# GET isteği atılarak ilgili veriler çekilsin
def get_news(parameters, headers={}):
    conn = http.client.HTTPSConnection("seffaflik.epias.com.tr")
    conn.request("GET", parameters)
    res = conn.getresponse()
    data = res.read()
    return data.decode("utf-8")

In [21]:
# API servisine,içinde bulunduğumuz gün 2022 05 28 formatında endDate ve startDate parametresi verildi
page = get_news(
        parameters="/transparency/service/market/intra-day-trade-history?endDate=2022-05-28&startDate=2022-05-28")
page

'{"resultCode":"0","resultDescription":"success","body":{"intraDayTradeHistoryList":[{"id":600992350,"date":"2022-05-28T00:00:14.000+0300","conract":"PH22052802","price":1989,"quantity":25},{"id":600992504,"date":"2022-05-28T00:00:47.000+0300","conract":"PH22052802","price":1989,"quantity":1},{"id":600992509,"date":"2022-05-28T00:00:47.000+0300","conract":"PH22052802","price":1989,"quantity":3},{"id":600992654,"date":"2022-05-28T00:00:48.000+0300","conract":"PH22052802","price":1988.99,"quantity":10},{"id":600993007,"date":"2022-05-28T00:00:48.000+0300","conract":"PH22052802","price":1989,"quantity":1},{"id":600993113,"date":"2022-05-28T00:00:48.000+0300","conract":"PH22052802","price":1989,"quantity":3},{"id":600994409,"date":"2022-05-28T00:02:07.000+0300","conract":"PH22052818","price":2762.13,"quantity":40},{"id":600994237,"date":"2022-05-28T00:02:08.000+0300","conract":"PH22052818","price":2762.12,"quantity":129},{"id":600994521,"date":"2022-05-28T00:02:08.000+0300","conract":"PH22

In [22]:
# conract değerlerine göre gruplanması ve ortalamanın alınması 
df_groupby=df.groupby(["conract"]).mean()
df_groupby=df_groupby.reset_index()
   
# toplam işlem tutarını tutan liste
list_toplam_islem_tutari1=[]
toplam_islem_tutari1=0
# toplam işlem miktarını tutan liste
list_toplam_islem_miktari1=[]
toplam_islem_miktari1=0
# Ağırlıklı ortalama fiyatını tutan liste
list_agirlikli_ortalama_fiyat1=[]
agirlikli_ortalama_fiyat1=0
#Conract değerinin PH ’dan sonra gelen 2 karakterler sırasıyla yıl,ay,gün ve saati temsil eder.
#Bu contract değerini iki iki ayırarak tarih ve saat değerlerini tutan liste
list_tarih1=[]
tarih1=""

for i in range(0,len(df_groupby)):
    #Conract değeri sadece PH ile başlayanlar için işlem yapılacaktır.    
    if 'PH' in df_groupby["conract"][i]:
      
        toplam_islem_tutari1=df_groupby["price"][i]*df_groupby["quantity"][i]/10
        list_toplam_islem_tutari1.append(toplam_islem_tutari1)
        
        toplam_islem_miktari1=df_groupby["quantity"][i]/10
        list_toplam_islem_miktari1.append(toplam_islem_miktari1)
        
        agirlikli_ortalama_fiyat1=toplam_islem_tutari1/toplam_islem_miktari1
        list_agirlikli_ortalama_fiyat1.append(agirlikli_ortalama_fiyat1)
        
        #Conract değerinin PH ’dan sonra gelen 2 karakteri sırayla yıl ay gün ve saat olarak ayır.
        tarih1=df_groupby["conract"][i][6]+df_groupby["conract"][i][7]+"/"+df_groupby["conract"][i][4]+df_groupby["conract"][i][5]+"/20"+df_groupby["conract"][i][2]+df_groupby["conract"][i][3]+" "+df_groupby["conract"][i][8]+df_groupby["conract"][i][9]+":00"
        list_tarih1.append(tarih1)
        

index=len(list_toplam_islem_tutari1)
# yeni bir dataframe oluşturma
def new_dataframe(column_name,liste):
    return pd.DataFrame(columns=column_name,data=liste,index=range(index))


data1= new_dataframe(["Tarih"],list_tarih1)
data2= new_dataframe(["Toplam İşlem Miktarı(MWh)"],list_toplam_islem_miktari1)
data3= new_dataframe(["Toplam İşlem Tutarı(TL)"],list_toplam_islem_tutari1)
data4= new_dataframe(["Ağırlıklı Ortalama Fiyat(TL/MWh)"],list_agirlikli_ortalama_fiyat1)
#dataframeleri birliştirme
df_new= pd.concat([data1,data2,data3,data4], axis = 1)
df_new


,Tarih,Toplam İşlem Miktarı(MWh),Toplam İşlem Tutarı(TL),Ağırlıklı Ortalama Fiyat(TL/MWh)
0,26/01/2022 02:00,3.463830,2861.647401,826.151277
1,26/01/2022 03:00,2.820482,2051.380682,727.315663
2,26/01/2022 04:00,3.908805,2852.231677,729.694025
3,26/01/2022 05:00,3.135652,2272.802495,724.826087
4,26/01/2022 06:00,1.765152,1515.670616,858.663182
5,26/01/2022 07:00,1.857447,2063.393397,1110.876170
6,26/01/2022 08:00,3.128671,4256.845049,1360.591958
7,26/01/2022 09:00,3.350382,4618.814188,1378.593435
8,26/01/2022 10:00,3.928105,5418.928862,1379.527647
9,26/01/2022 11:00,3.786667,5228.105802,1380.661744


In [20]:
# conract değerlerine göre gruplanmadan yapılması

# toplam işlem tutarını tutan liste
list_toplam_islem_tutari=[]
toplam_islem_tutari=0
# toplam işlem miktarını tutan liste
list_toplam_islem_miktari=[]
toplam_islem_miktari=0
# Ağırlıklı ortalama fiyatını tutan liste
list_agirlikli_ortalama_fiyat=[]
agirlikli_ortalama_fiyat=0
#Conract değerinin PH ’dan sonra gelen 2 karakterler sırasıyla yıl,ay,gün ve saati temsil eder.
#Bu contract değerini iki iki ayırarak tarih ve saat değerlerini tutan liste
list_tarih=[]
tarih=""
for i in range(0,len(df)):
    #Conract değeri sadece PH ile başlayanlar için işlem yapılacaktır.    
    if 'PH' in df["conract"][i]:
        
        toplam_islem_tutari=df["price"][i]*df["quantity"][i]/10
        list_toplam_islem_tutari.append(toplam_islem_tutari)
        
        toplam_islem_miktari=df["quantity"][i]/10
        list_toplam_islem_miktari.append(toplam_islem_miktari)
        
        agirlikli_ortalama_fiyat=toplam_islem_tutari/toplam_islem_miktari
        list_agirlikli_ortalama_fiyat.append(agirlikli_ortalama_fiyat)
        
        #Conract değerinin PH ’dan sonra gelen 2 karakteri sırayla yıl ay gün ve saat olarak ayır.
        tarih=df["conract"][i][6]+df["conract"][i][7]+"/"+df["conract"][i][4]+df["conract"][i][5]+"/20"+df["conract"][i][2]+df["conract"][i][3]+" "+df["conract"][i][8]+df["conract"][i][9]+":00"
        list_tarih.append(tarih)

index=len(list_toplam_islem_tutari)
# yeni bir dataframe oluşturma
def new_dataframe(column_name,liste):
    return pd.DataFrame(columns=column_name,data=liste,index=range(index))


data1= new_dataframe(["Tarih"],list_tarih)
data2= new_dataframe(["Toplam İşlem Miktarı(MWh)"],list_toplam_islem_miktari)
data3= new_dataframe(["Toplam İşlem Tutarı(TL)"],list_toplam_islem_tutari)
data4= new_dataframe(["Ağırlıklı Ortalama Fiyat(TL/MWh)"],list_agirlikli_ortalama_fiyat)
#dataframeleri birliştirme
df_new= pd.concat([data1,data2,data3,data4], axis = 1)
df_new

,Tarih,Toplam İşlem Miktarı(MWh),Toplam İşlem Tutarı(TL),Ağırlıklı Ortalama Fiyat(TL/MWh)
0,26/01/2022 03:00,0.5,365.995,731.99
1,26/01/2022 03:00,0.5,366.000,732.00
2,26/01/2022 03:00,0.5,365.995,731.99
3,26/01/2022 08:00,1.0,1341.990,1341.99
4,26/01/2022 08:00,1.0,1341.980,1341.98
...,...,...,...,...
6748,27/01/2022 01:00,1.7,2337.534,1375.02
6749,27/01/2022 07:00,0.1,137.101,1371.01
6750,27/01/2022 08:00,0.1,137.040,1370.40
6751,27/01/2022 04:00,1.8,2412.018,1340.01
